In [ ]:
%load_ext autoreload
%autoreload 2
from lewidi_lib import enable_logging

enable_logging()

# Remove Dictionary Definition
Applies to CSC and MP. Means comparing 32B on templates 31 (includes def) and 32

In [ ]:
import duckdb
from lewidi_lib import preds_file, list_preds

preds_fdf = list_preds().query(
    "dataset in ['CSC', 'MP'] and model_id.str.contains('32B') and template_id.isin(['31', '32']) and run_name == 'allex_10loops' and split=='train'"
)
rdf = duckdb.sql(
    f"SELECT * FROM read_parquet({[str(f) for f in preds_fdf['preds_file']]})"
).df()

In [ ]:
from lewidi_lib import process_rdf_and_add_perf_metrics

joint_df = process_rdf_and_add_perf_metrics(rdf, discard_invalid_pred=True)

In [ ]:
joint_df.groupby(["dataset", "template_id"]).size()

In [ ]:
joint_df.groupby(["dataset", "template_id"])["ws_loss"].mean()

In [ ]:
from lewidi_lib import bootstrap_avg


cis = joint_df.groupby(["dataset", "template_id"])["ws_loss"].agg(
    lambda xs: bootstrap_avg(xs, method="basic", batch=1000)
)

In [ ]:
export_df = pd.concat(
    [
        cis.reset_index(),
        pd.json_normalize(cis.reset_index()["ws_loss"].apply(lambda x: x.to_dict())),
    ],
    axis=1,
).round(3)
export_df.to_csv("tables/ablation-rm-def.csv", index=False)

# Remove Perspectivist Prompt
Applies to all datasets. Compares 32B on templates 31 (includes perspectivist prompt) and 3 (no perspectivist prompt).

In [ ]:
from lewidi_lib import list_preds
import pandas as pd

preds_files_df = list_preds().query(
    "model_id.str.contains('32B') and template_id.isin(['31', '3']) and run_name == 'allex_10loops' and split=='train'"
)
preds_files_df

In [ ]:
rdf = duckdb.sql(
    f"SELECT * FROM read_parquet({[str(f) for f in preds_files_df['preds_file']]})"
).df()
joint_df = process_rdf_and_add_perf_metrics(rdf, discard_invalid_pred=True)

In [ ]:
cis = joint_df.groupby(["dataset", "template_id"])["ws_loss"].agg(
    lambda xs: bootstrap_avg(xs, method="basic", batch=1000)
)

In [ ]:
export_df = pd.concat(
    [
        cis.reset_index(),
        pd.json_normalize(cis.reset_index()["ws_loss"].apply(lambda x: x.to_dict())),
    ],
    axis=1,
).round(3)
export_df.to_csv("tables/ablation-rm-perspectivist-prompt.csv", index=False)